# Write year-2000 glacier area out
We need the initial glacierized area of each basin to compute the glacier-modified SPEI.  This code should read in glacier area series from each model and extract the year-2000 value for the "initial" condition.  (Note that GloGEM simulations start in 1980, so these won't all be the first value in the series.)

29 Jun 2023 | EHU

In [51]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import itertools
import json
import os
import glob

## GloGEM data

In [10]:
## Generic the filepath to the main data folder
fpath_glogem = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/GloGEM-output/RGI11-CentralEurope/files/'

#All of the climate models used
modelnames = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0']

SSPpaths = ['ssp126','ssp245','ssp370','ssp585']   #Specifiying the SSP

alpine_basins = {'RHINE':'6242', 
                 'RHONE':'6243',
                 'PO':'6241', 
                 'DANUBE':'6202'} ## GRDC Major River Basin identifiers for the 4 alpine basins we can study

In [3]:
all_areas = [[] for _ in SSPpaths]

for s, SSPpath in enumerate(SSPpaths):
    model_areas = []
    for modelname in modelnames:
        temp_df = pd.read_csv(fpath_glogem + modelname  + '/' + SSPpaths[s]  + '/' + 'centraleurope_Area_r1.dat', sep='\s+', index_col="ID")
        model_areas.append(temp_df)
    all_areas[s] = model_areas

For the period 1980-2000, the forcing is all ERA5 + noise (per Matthias Huss's email) -- thus we can select only the first of the SSP/GCM combinations and find the year-2000 value there

In [15]:
baseline_area = all_areas[0][0]

In [16]:
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    fpath = '/Users/lizz/Documents/Research/Runoff-intercomparison/msc_thesis-multi_gcm/code/data/rgi_ids_per_basin.json' ## correct for local run
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [47]:
def sum_basin_area(basin_RGI_list, area_data):
    # Create new list to match our RGI formatting
    new_basin_list = [int(str(x)[-4:]) for x in basin_RGI_list]
    
    # Filter new_basin_list to keep only the indexes present in the DataFrame
    new_basin_list = [x for x in new_basin_list if x in area_data.index]
    
    # Extract glaciers contained in the list from original df and create a new df
    new_df = area_data.loc[new_basin_list].copy()
    # print(new_df)
    
    # Sum the values of the glaciers within the basin
    summed_basin_area = new_df.sum()
    #print(summed_basin_runoff)
    
    return summed_basin_area

In [38]:
area_sum = sum_basin_area(select_glaciers_json(alpine_basins['RHONE']), baseline_area)

        1980    1981    1982    1983    1984    1985    1986    1987    1988  \
ID                                                                             
1238  15.823  15.956  16.016  15.999  16.100  16.272  16.187  16.200  16.183   
1299   0.014   0.016   0.017   0.019   0.022   0.024   0.025   0.025   0.027   
1324   0.034   0.038   0.037   0.038   0.040   0.042   0.043   0.043   0.043   
1325   0.051   0.054   0.053   0.053   0.056   0.061   0.059   0.056   0.057   
1326   0.037   0.041   0.045   0.046   0.049   0.052   0.050   0.052   0.052   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
3816   1.476   1.507   1.532   1.528   1.589   1.599   1.591   1.588   1.569   
3817   1.759   1.797   1.804   1.807   1.908   1.975   1.961   1.959   1.929   
3818   0.016   0.019   0.019   0.020   0.022   0.025   0.025   0.025   0.026   
3819   0.011   0.013   0.013   0.013   0.014   0.016   0.016   0.016   0.016   
3820   0.015   0.016   0.017   0.016   0

In [39]:
area_sum

1980    915.638
1981    931.161
1982    938.276
1983    935.805
1984    952.420
         ...   
2096    365.693
2097    364.258
2098    362.240
2099    361.975
2100    347.686
Length: 121, dtype: float64

In [43]:
area_sum.loc['2000']

956.327

In [48]:
basin_areas_glogem = {k: sum_basin_area(select_glaciers_json(alpine_basins[k]), baseline_area).loc['2000']
                      for k in alpine_basins.keys()}

In [49]:
basin_areas_glogem ## in km2

{'RHINE': 364.017, 'RHONE': 956.327, 'PO': 335.249, 'DANUBE': 429.959}

## PyGEM area sum

In [52]:
#All of the climate models used in PyGEM
modelnames_py = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

#Generic filepath to navigate to Drive folder 
fpathPy = '/Volumes/GoogleDrive/My Drive/Runoff-intercomparison/PyGEM/11'

In [53]:
basin_gls = {}
for basin, code in alpine_basins.items():
    basin_gls[basin] = select_glaciers_json(code)

In [66]:
#Importing all area data for the region

fpath1 = '/area_annual-ssp585/R11_area_annual_c2_ba1_1set_2000_2100--' ##not separated by SSP, unlike runoff
file_pattern = f'{fpathPy + fpath1}*.nc'
file_list = glob.glob(file_pattern)

datasets = []  # Create an empty list for each SSP
if file_list: ## if there are files available
    for file in file_list:
        with xr.open_dataset(file) as ds:
            ds = ds.glac_area_annual.load()
            datasets.append(ds)

    combined_ds = xr.concat(datasets, dim='glacier')  # Concatenate the datasets
    area_pygem_km2 = 1e-6*combined_ds ## convert to km2

In [67]:
area_pygem_km2

<xarray.DataArray 'glac_area_annual' (model: 12, glacier: 3925, year: 102)>
array([[[0.073     , 0.07237955, 0.07226154, ..., 0.        ,
         0.        , 0.        ],
        [0.018     , 0.01754999, 0.01747182, ..., 0.        ,
         0.        , 0.        ],
        [0.053     , 0.05261363, 0.05258091, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.01      , 0.00986962, 0.00983899, ..., 0.        ,
         0.        , 0.        ],
        [0.013     , 0.01231105, 0.01222957, ..., 0.        ,
         0.        , 0.        ],
        [0.015     , 0.01446044, 0.01447216, ..., 0.        ,
         0.        , 0.        ]],

       [[0.073     , 0.07292026, 0.07279155, ..., 0.        ,
         0.        , 0.        ],
        [0.018     , 0.01791043, 0.01784415, ..., 0.        ,
         0.        , 0.        ],
        [0.053     , 0.05294587, 0.05285433, ..., 0.        ,
         0.        , 0.        ],
...
        [0.01      , 0.00997466, 0.00994828, ..., 0.        ,
         0.        , 0.        ],
        [0.013     , 0.01288535, 0.0127606 , ..., 0.        ,
         0.        , 0.        ],
        [0.015     , 0.01500411, 0.01506111, ..., 0.        ,
         0.        , 0.        ]],

       [[0.073     , 0.072833  , 0.07275193, ..., 0.        ,
         0.        , 0.        ],
        [0.018     , 0.01785446, 0.01783609, ..., 0.        ,
         0.        , 0.        ],
        [0.053     , 0.05283624, 0.05940383, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.01      , 0.00998377, 0.00991842, ..., 0.        ,
         0.        , 0.        ],
        [0.013     , 0.01292994, 0.01264863, ..., 0.        ,
         0.        , 0.        ],
        [0.015     , 0.0149467 , 0.01479517, ..., 0.        ,
         0.        , 0.        ]]])
Coordinates:
    RGIId          (glacier) object 'RGI60-11.01001' ... 'RGI60-11.03927'
    Climate_Model  (model) object 'BCC-CSM2-MR' 'CESM2' ... 'NorESM2-MM'
    lon            (glacier) float64 10.03 8.963 8.975 ... 13.45 13.45 13.47
    lat            (glacier) float64 46.79 46.79 46.79 ... 46.37 46.37 46.37
  * model          (model) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * glacier        (glacier) int64 1 2 3 4 5 6 7 ... 919 920 921 922 923 924 925
  * year           (year) int64 2000 2001 2002 2003 2004 ... 2098 2099 2100 2101

In [276]:
# Sorting into basins
### edit here - modify from CentralEurope-3ModelAgg
basin_datasets = {}
for basin, glacier_list in basin_gls.items():
    ## loop over them all, drop the irrelevant IDs, and concatenate the result
    basin_datasets[basin] = {}
    for s, SSP in enumerate(SSPs):
        ds_list = []
        try:
            ds_filtered = annual_rf_ds[SSP].where(annual_rf_ds[SSP].RGIId.isin(glacier_list), drop=True)
            #print(ds_filtered)
            ds_list.append(ds_filtered)
        except ValueError: ## happens if there are no glaciers from this batch in the selected region
            continue
        basin_datasets[basin][SSP] = xr.concat(ds_list, dim='glacier')

In [277]:
#Flipping indexing (to match other models) and summing basins
basin_sums_py = {}
for s, SSP in enumerate(SSPs):        
    basin_sums_py[SSP] = {}
    for basin, glacier_list in basin_gls.items():
        basin_sums_py[SSP][basin] = basin_datasets[basin][SSP].sum(dim='glacier')